# Training data processing for KNOAP2020 challenge
This example shows how to create an outcome variable for predicting incident symptomatic radiographic knee osteoarthritis using OAI data. Following this example, you can create a training data for the KNOAP2020 challenge.<br><br>
For this example, you need to download OAICompleteData in ASCII format from the OAI website and update the 'OAI_data_path' with the path where the OAI radiographical scores and clinical data are (text files).<br>
<br>
The files with the name kxr_sq_buXX.txt (XX=follow-up time point) contain KL grades and osteophyte grades for the knees.<br>
The files with the name AllClinicalXX.txt (XX=follow-up time point) contain the clinical variables.<br>
<br>
To identify which knees will develop symptomatic radiographic knee osteoarthritis within the 72 months follow-up in the OAI data we need need to create an outcome variable. We will first create a variable to describe if the knee has or does not have a symptomatic knee osteoarthritis. For this we need:<br>
- Osteophyte scores
- Symptoms assessed with the question: "Did you experience symptoms in your knee most days in the past 30 days?" (yes/no).

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def get_radiographical_scores(data_path,timepoint):
    """
    Extracts and combines radiological scores from the OAI time points. OAI has two different 
    projects with KL grades (project 15 and 37/42)

    Parameters
    ----------
    data_path : OAI data path
    timepoint : 00, 01, 03, 05, 06, 08

    Returns
    ----------
    OST_data : dataframe
    ----------
    
    Notes
    ----------
    Osteophyte score variable names:
    baseline: V00XROSFM, V00XROSTM, V00XROSFL, V00XROSTL
    12 months: V01XROSFM, V01XROSTM, V01XROSFL, V01XROSTL
    24 months: V03XROSFM, V03XROSTM, V03XROSFL, V03XROSTL
    36 months: V05XROSFM, V05XROSTM, V05XROSFL, V05XROSTL
    48 months: V06XROSFM, V06XROSTM, V06XROSFL, V06XROSTL
    72 months: V08XROSFM, V08XROSTM, V08XROSFL, V08XROSTL

    KL grade variable names:
    V00XRKL, V01XRKL, V03XRKL, v05XRKL, V06XRKL, V08XRKL

    """

    for timepoint in timepoints:
        xray_data = pd.read_csv(os.path.join(data_path,"kxr_sq_bu"+str(timepoint)+".txt"), sep="|")
        if timepoint in ['01','03','05']:
            project = 'readprj'
        else:
            project = 'READPRJ'

        xray_scores_proj15 = xray_data.loc[xray_data[project]==15,:].copy()
        xray_scores_proj37 = xray_data.loc[(xray_data[project]==37)|(xray_data[project]==42),:].copy()

        # variable names
        med_fem_OST = 'V'+timepoint+'XROSFM'
        lat_fem_OST = 'V'+timepoint+'XROSFL'
        med_tib_OST = 'V'+timepoint+'XROSTM'
        lat_tib_OST = 'V'+timepoint+'XROSTL'
        OST_score_name = 'OST'+timepoint
        if timepoint =='05': # KL variable starts with small letter in time point 05
            KL_variable = 'v'+timepoint+'XRKL'
        else:
            KL_variable = 'V'+timepoint+'XRKL'

        # combine osteophyte scores from different location into one variable
        xray_scores_proj37[OST_score_name]=xray_scores_proj37[med_fem_OST] + xray_scores_proj37[lat_fem_OST] + xray_scores_proj37[med_tib_OST] + xray_scores_proj37[lat_tib_OST]
        number_of_OST_knees_proj37 = len(xray_scores_proj37[xray_scores_proj37[OST_score_name]>0])
        number_of_control_knees_proj37 = len(xray_scores_proj37[xray_scores_proj37[OST_score_name]==0])
#        print("{0} variable in project 37 has {1} knees without and {2} knees with osteophytes. Number of no/yes knees: {3}".format(OST_score_name,number_of_control_knees_proj37,number_of_OST_knees_proj37,number_of_control_knees_proj37+number_of_OST_knees_proj37))

        if timepoint != '08':
            # combine osteophyte scores from different location into one variable
            xray_scores_proj15[OST_score_name]=xray_scores_proj15[med_fem_OST] + xray_scores_proj15[lat_fem_OST] + xray_scores_proj15[med_tib_OST] + xray_scores_proj15[lat_tib_OST]
            number_of_OST_knees_proj15 = len(xray_scores_proj15[xray_scores_proj15[OST_score_name]>0])
            number_of_control_knees_proj15 = len(xray_scores_proj15[xray_scores_proj15[OST_score_name]==0])
#            print("{0} variable in project 15 has {1} knees without and {2} knees with osteophytes. Number of no/yes knees: {3}".format(OST_score_name,number_of_control_knees_proj15,number_of_OST_knees_proj15,number_of_control_knees_proj15+number_of_OST_knees_proj15))      

            # combine KL and osteophyte scores. If both projects have scores for the same knee, use the highest grade.
            xray_scores_proj15 = combine_KL_grades(timepoint,KL_variable,xray_scores_proj15,xray_scores_proj37)
            OST_scores = combine_OST_grades(timepoint,OST_score_name,KL_variable,xray_scores_proj15,xray_scores_proj37)
        else: # timepoint=='08', Project 15 does not have scores for 72 months data
            OST_scores = xray_scores_proj37[['ID','SIDE',OST_score_name,KL_variable]].copy()
            # If OST score is missing and KL = 0, set OST score = 0
            OST_scores.loc[(np.isnan(OST_scores[OST_score_name])==True)&(OST_scores[KL_variable]==0),OST_score_name]=0

        OST0_cases = len(OST_scores[OST_scores[OST_score_name]==0])
        OST1_cases = len(OST_scores[OST_scores[OST_score_name]>0])
        print("{0} variable has {1} knees without and {2} knees with osteophytes. Number of no/yes knees: {3}".format(OST_score_name,OST0_cases,OST1_cases,OST0_cases+OST1_cases))

        if timepoint == '00':
            OST_data = OST_scores.copy()
        else:
            OST_data = pd.merge(OST_data,OST_scores, how='left', on=['ID','SIDE'],sort=True)    
        
    return OST_data

In [3]:
def combine_KL_grades(timepoint,KL_variable,xray_scores_proj15,xray_scores_proj37):
    """
    OAI has two different projects with KL grades. This function combines them and takes the highest KL grade. 

    Parameters
    ----------
    timepoint : 00, 01, 03, 05, 06, 08
    KL_variable : KL variable name at the specific timepoint
    xray_scores_proj15 : Dataframe of X-ray scores in project 15
    xray_scores_proj37 : Dataframe of X-ray scores in project 37

    Returns
    ----------
    xrays_scores_proj15 : dataframe    
    
    """   
    
    # Iterate through cases in proj37. Select corresponding knee from proj15.
    for index, row in xray_scores_proj37.iterrows():
        ID_proj37 = row['ID']
        Side_proj37 = row['SIDE']
        KL_proj37 = row[KL_variable]
        row_proj15 = xray_scores_proj15.loc[(xray_scores_proj15['ID'] == ID_proj37)&(xray_scores_proj15['SIDE'] == Side_proj37)]
        # Check if the knee is missing from the project15
        if len(row_proj15)==0: 
            # Check if the KL grade exists in project 37/42
            if np.isnan(KL_proj37)==False: 
                # Append xray_scores_proj15 with the KL in project 37/42
                xray_scores_proj15 = xray_scores_proj15.append(row[['ID','SIDE',KL_variable]],ignore_index=True)
#            else:
#                pass
        else: # the knee exist in the project15
            KL_proj15 = row_proj15[KL_variable].values[0]
            if (np.isnan(KL_proj37)==False)&(np.isnan(KL_proj15)==False):
                # Check if KL in project 37 is higher than in project 15.
                if KL_proj37 > KL_proj15:
                    # Use the KL from the project 37/42
                    xray_scores_proj15.loc[row_proj15.index,KL_variable]=KL_proj37
#                elif KL_p37 < KL_p15:
#                    pass
            elif (np.isnan(KL_proj37)==False)&(np.isnan(KL_proj15)==True):
                # Use the KL from the project 37/42
                xray_scores_proj15.loc[row_proj15.index,KL_variable]=KL_proj37
#            elif (np.isnan(KL_p37)==True) & (np.isnan(KL_p15)==False):
#                pass
#            elif (np.isnan(KL_p37)==True)&(np.isnan(KL_p15)==True):
#                pass
            
    return xray_scores_proj15

In [4]:
def combine_OST_grades(timepoint,OST_variable,KL_variable,xray_scores_proj15,xray_scores_proj37):
    """
    OAI has two different projects with OST grades. This function combines them and takes the highest OST grade. 
    
    Parameters
    ----------
    timepoint : 00, 01, 03, 05, 06, 08
    OST_variable : OST variable name at the specific timepoint
    KL_variable : KL variable name at the specific timepoint
    xray_scores_proj15 : Dataframe of X-ray scores in project 15
    xray_scores_proj37 : Dataframe of X-ray scores in project 37

    Returns
    ----------
    OST_scores : dataframe 
    
    """
    
    OST_scores = xray_scores_proj15[['ID','SIDE',OST_variable,KL_variable]].copy()
    
    # If KL=0 and OST=NaN, set OST=0
    OST_scores.loc[(np.isnan(OST_scores[OST_variable])==True)&(OST_scores[KL_variable]==0),OST_variable]=0
    for index, row in xray_scores_proj37.iterrows():
        ID_proj37 = row['ID']
        Side_proj37 = row['SIDE']
        OST_proj37 = row[OST_variable]
        row_proj15 = OST_scores.loc[(OST_scores['ID']==ID_proj37)&(OST_scores['SIDE']==Side_proj37)]
        # Check if the knee is missing from the project15
        if len(row_proj15)==0:
            # Check if the project 37/42 has OST grade for this knee
            if np.isnan(OST_proj37)==False:
                # Append OST_scores with the OST score in project 37/42
                OST_scores = OST_scores.append(row[['ID','SIDE',OST_variable]],ignore_index=True)
#            else:
#                pass
        else: # the knee exist in the project15
            OST_proj15 = row_proj15[OST_variable].values[0]
            if (np.isnan(OST_proj37)==False)&(np.isnan(OST_proj15)==False):
                 # Check if OST in project 37 is higher than in project 15.
                if OST_proj37 > OST_proj15:
                    # Use the OST score from the project 37/42
                    OST_scores.loc[row_proj15.index,OST_variable]=OST_proj37
#                elif OST_p37 < OST_p15:
#                    pass
            elif (np.isnan(OST_proj37)==False)&(np.isnan(OST_proj15)==True):
                # Use the OST score from the project 37/42
                OST_scores.loc[row_proj15.index,OST_variable]=OST_proj37
#            elif (np.isnan(OST_p37)==True) & (np.isnan(OST_p15)==False):
#                pass    
#            elif (np.isnan(OST_p37)==True)&(np.isnan(OST_p15)==True):
#                pass 

    OST_scores[['ID','SIDE']] = OST_scores[['ID','SIDE']].astype(int)
    
    return OST_scores

In [5]:
def get_pain_scores(data_path,timepoints):
    """
    Parameters
    ----------
    data_path : OAI data path
    timepoints : 00, 01, 03, 05, 06, 08

    Returns
    ----------
    pain_data : dataframe 

    
    Notes
    ----------
    Symptoms were assessed with the question: "Did you experience symptoms in your knee most days 
    in the past 30 days?" (yes/no).
    
    Variable names for symptoms:
    Baseline: P01KPR30CV, P01KPL30CV
    12 months: V01KPR30CV, V01KPL30CV
    24 months: V03KPR30CV, V03KPL30CV
    36 months: V05KPR30CV, V05KPL30CV
    48 months: V06KPR30CV, V06KPL30CV
    72 months: V08KPR30CV, V08KPL30CV
    """
    
    for timepoint in timepoints:
        clinical_data = pd.read_csv(os.path.join(data_path,"AllClinical"+timepoint+".txt"), sep="|")
        pain_variable_name = 'Pain'+timepoint
        if timepoint == '00':
            pain_variable_right = 'P01KPR30CV'
            pain_variable_left = 'P01KPL30CV'
        else:
            pain_variable_right = 'V'+timepoint+'KPR30CV'
            pain_variable_left = 'V'+timepoint+'KPL30CV'

        pain_r = clinical_data[['ID',pain_variable_right]].copy()
        pain_r.insert(1,'SIDE',1) # create 'SIDE' variable and put 1 to right side knees
        pain_r=pain_r.rename(columns={pain_variable_right:pain_variable_name})
        pain_l = clinical_data[['ID',pain_variable_left]].copy()
        pain_l.insert(1,'SIDE',2) # create 'SIDE' variable and put 2 to left side knees
        pain_l=pain_l.rename(columns={pain_variable_left:pain_variable_name})
        pain = pd.concat([pain_r,pain_l])
        # Harmonize pain coding, because it is coded differently for baseline (0 months) and 12-months data
        if timepoint in ['00','01']: 
            pain.loc[pain[pain_variable_name]=='0: No',pain_variable_name]=0
            pain.loc[pain[pain_variable_name]=='1: Yes',pain_variable_name]=1
            pain.loc[(pain[pain_variable_name]==0)|(pain[pain_variable_name]==1)==False,pain_variable_name]='NaN'

        number_of_pain_knees = len(pain[pain[pain_variable_name]==1])
        number_of_control_knees = len(pain[pain[pain_variable_name]==0])
        print("{0} variable has {1} knees without pain and {2} knees with pain. Number of no/yes knees: {3}".format(pain_variable_name,number_of_control_knees,number_of_pain_knees,number_of_control_knees+number_of_pain_knees))
        
        if timepoint =='00':
            pain_data = pain.copy()
        else:
            pain_data = pd.merge(pain_data,pain,how='left', on=['ID','SIDE'],sort=True)
        
    return pain_data

In [6]:
def get_TKR_data(OAI_data_path, OAI_train):
    """
    Checks if TKR is done within the follow-up. If yes, we assume that the knee has symptomatic OA.
    
    Parameters
    ----------
    OAI_data_path : OAI data path
    OAI_train : dataframe with the variables

    Returns
    ----------
    OAI_train : dataframe 


    Notes
    ----------
    V99ERKXRPR/V99ELKXRPR=visit with X-ray prior TKR
    """

    outcomes_data = pd.read_csv(os.path.join(OAI_data_path,"Outcomes99.txt"), sep="|")
    outcomes_r = outcomes_data[['id','V99ERKXRPR']].copy()
    outcomes_r.insert(1,'SIDE',1)
    outcomes_r=outcomes_r.rename(columns={'id':'ID','V99ERKXRPR':'TKR_timepoint'})
    outcomes_l = outcomes_data[['id','V99ELKXRPR']].copy()
    outcomes_l.insert(1,'SIDE',2)
    outcomes_l=outcomes_l.rename(columns={'id':'ID','V99ELKXRPR':'TKR_timepoint'})
    outcomes = pd.concat([outcomes_r, outcomes_l],sort=False)
    OAI_train = pd.merge(OAI_train, outcomes, how='left', on=['ID','SIDE'],sort=True)
    
    return OAI_train

In [7]:
def get_demographics(OAI_data_path, OAI_train):
    """
    Parameters
    ----------
    OAI_data_path : OAI data path
    OAI_train : dataframe with the variables

    Returns
    ----------
    OAI_train : dataframe 
    """    
    
    # get gender from the Enrollees.txt file
    demographics_data = pd.read_csv(os.path.join(OAI_data_path,"Enrollees.txt"), sep="|")
    gender_r = demographics_data[['ID','P02SEX']].copy()
    gender_r.insert(1,'SIDE',1)
    gender_l = demographics_data[['ID','P02SEX']].copy()
    gender_l.insert(1,'SIDE',2)
    gender = pd.concat([gender_r, gender_l])
    
    # get age and BMI data at baseline from the AllClinical00.txt file
    clinical_data = pd.read_csv(os.path.join(OAI_data_path,"AllClinical00.txt"), sep="|")
    age_BMI_r = clinical_data[['ID','V00AGE','P01BMI']].copy()
    age_BMI_r.insert(1,'SIDE',1)
    age_BMI_l = clinical_data[['ID','V00AGE','P01BMI']].copy()
    age_BMI_l.insert(1,'SIDE',2)
    age_BMI = pd.concat([age_BMI_r, age_BMI_l])
    
    demographics = pd.merge(age_BMI, gender, how='left', on=['ID','SIDE'],sort=True)
    OAI_train = pd.merge(OAI_train, demographics, how='left', on=['ID','SIDE'],sort=True)
    
    return OAI_train

In [8]:
# Add here the path where the OAI radiographical scores and clinical data are (text files)
OAI_data_path = "./OAICompleteData_ASCII/"
timepoints = ['00','01','03','05','06','08']

# get osteophyte data
OST_data = get_radiographical_scores(OAI_data_path,timepoints)
# get pain symptoms data
pain_data = get_pain_scores(OAI_data_path,timepoints)

# Combine pain and osteophyte data in to one dataframe
OAI_train = pd.merge(pain_data, OST_data, how='left', on=['ID','SIDE'],sort=True)

# Combine ID and side
OAI_train['Knee']=''
OAI_train.loc[OAI_train['SIDE']==1,'Knee']=OAI_train['ID'].astype(str)+'_R'
OAI_train.loc[OAI_train['SIDE']==2,'Knee']=OAI_train['ID'].astype(str)+'_L'

# Create a variable for symptomatic ROA at each time point
for timepoint in timepoints:
    OAI_train['ACR'+timepoint]='NaN'
    OAI_train.loc[(OAI_train['Pain'+timepoint]==0)|(OAI_train['OST'+timepoint]==0),'ACR'+timepoint]=0
    OAI_train.loc[(OAI_train['Pain'+timepoint]==1)&(OAI_train['OST'+timepoint]>0),'ACR'+timepoint]=1

# get information about the total knee replacements within the follow-up
OAI_train = get_TKR_data(OAI_data_path, OAI_train)

# get the demographics of the subjects
OAI_train = get_demographics(OAI_data_path, OAI_train)

# create variable for incident radiographic knee OA within the follow-up
OAI_train['iSROA']='NaN'
OAI_train.loc[(OAI_train['ACR00']==0)&(OAI_train['V00XRKL']<2)&(OAI_train['ACR08']==0),'iSROA']=0
OAI_train.loc[(OAI_train['ACR00']==0)&(OAI_train['V00XRKL']<2)&((OAI_train['ACR01']==1)|(OAI_train['ACR03']==1)|(OAI_train['ACR05']==1)|(OAI_train['ACR06']==1)|(OAI_train['ACR08']==1)),'iSROA']=1

# check if TKR is done during the follow-up
OAI_train.loc[(OAI_train['ACR00']==0)&(OAI_train['V00XRKL']<2)&((OAI_train['TKR_timepoint']=='1: 12-month')|(OAI_train['TKR_timepoint']=='3: 24-month')|(OAI_train['TKR_timepoint']=='5: 36-month')|(OAI_train['TKR_timepoint']=='6: 48-month')),'iSROA']=1
print("{0} knees without iSROA and {1} knees with iSROA.".format(len(OAI_train[OAI_train['iSROA']==0]),len(OAI_train[OAI_train['iSROA']==1])))

#TODO: Include the following variables:
# Varus malalignment, History of knee injury, Mild symptoms, Presence of Heberden nodes, 
# Joint line tenderness, Crepitus, Morning stiffness, and Postmenopausal status

OST00 variable has 3864 knees without and 4442 knees with osteophytes. Number of no/yes knees: 8306
OST01 variable has 3405 knees without and 4214 knees with osteophytes. Number of no/yes knees: 7619
OST03 variable has 3165 knees without and 3966 knees with osteophytes. Number of no/yes knees: 7131
OST05 variable has 2971 knees without and 3804 knees with osteophytes. Number of no/yes knees: 6775
OST06 variable has 2818 knees without and 3679 knees with osteophytes. Number of no/yes knees: 6497
OST08 variable has 1896 knees without and 308 knees with osteophytes. Number of no/yes knees: 2204
Pain00 variable has 6614 knees without pain and 2934 knees with pain. Number of no/yes knees: 9548
Pain01 variable has 6556 knees without pain and 2356 knees with pain. Number of no/yes knees: 8912
Pain03 variable has 6318 knees without pain and 2259 knees with pain. Number of no/yes knees: 8577
Pain05 variable has 6233 knees without pain and 2250 knees with pain. Number of no/yes knees: 8483
Pain0

In [9]:
OAI_gender_BMI_age_matched = OAI_train.loc[(OAI_train['P02SEX']=='2: Female')&(OAI_train['P01BMI']>=26.1)&(OAI_train['V00AGE']>=50)&(OAI_train['V00AGE']<=62)].copy()
OAI_gender_BMI_age_matched = OAI_gender_BMI_age_matched.loc[OAI_gender_BMI_age_matched['iSROA']!='NaN'].copy()
print("Gender, age, and BMI matched with the test set: {0} knees without iSROA and {1} knees with iSROA.".format(len(OAI_gender_BMI_age_matched[OAI_gender_BMI_age_matched['iSROA']==0]),len(OAI_gender_BMI_age_matched[OAI_gender_BMI_age_matched['iSROA']==1])))

Gender, age, and BMI matched with the test set: 444 knees without iSROA and 97 knees with iSROA.


In [10]:
OAI_train.to_csv('OAI_all_knees_outcome.csv', columns = ['Knee','iSROA'], index=False)
OAI_gender_BMI_age_matched.to_csv('OAI_KNOAP_matched_outcome.csv', columns = ['Knee','iSROA'], index=False)